In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmps9tybg8m/pubring.gpg' created
gpg: /tmp/tmps9tybg8m/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p my_drive
!google-drive-ocamlfuse my_drive

In [0]:
import xgboost as xgb
import numpy
import os
import pandas
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from scipy import stats

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [0]:
base = 'my_drive'+os.sep+'ML_Project'+os.sep

In [0]:
X = pandas.read_csv(base+"X_train.txt", delim_whitespace=True, header=None)
Y = pandas.read_csv(base+"Y_train.txt", delim_whitespace=True, header=None)
X_test = pandas.read_csv(base+"X_test.txt", delim_whitespace=True, header=None)

In [0]:
#X_main = X.values
#Y_main = Y.values
X = X.sample(n=10000,replace="False")
Y = Y.sample(n=10000,replace="False")
X_test = X_test.sample(n=10000,replace="False")

In [0]:
X = X.values
X_test = X_test.values
Y = Y.values

In [0]:
## Run if 'continuous' error comes
lbl = MinMaxScaler(copy=True, feature_range=(0, 1))
Y = lbl.fit_transform(Y)

In [0]:
xgb_model = xgb.XGBRegressor(verbose=1)
model_ = []
model_.append(('standardize', (MinMaxScaler(copy=True, feature_range=(0, 1)))))
model_.append(('xgb', xgb_model))
model__ = Pipeline(model_)
model__.get_params().keys()

dict_keys(['memory', 'steps', 'standardize', 'xgb', 'standardize__copy', 'standardize__feature_range', 'xgb__base_score', 'xgb__booster', 'xgb__colsample_bylevel', 'xgb__colsample_bytree', 'xgb__gamma', 'xgb__learning_rate', 'xgb__max_delta_step', 'xgb__max_depth', 'xgb__min_child_weight', 'xgb__missing', 'xgb__n_estimators', 'xgb__n_jobs', 'xgb__nthread', 'xgb__objective', 'xgb__random_state', 'xgb__reg_alpha', 'xgb__reg_lambda', 'xgb__scale_pos_weight', 'xgb__seed', 'xgb__silent', 'xgb__subsample', 'xgb__verbose'])

In [0]:
# RUN AGAIN DUDE. THERE WAS A MISTAKE I FIXED THE MISTAKE


xgb_model = xgb.XGBRegressor(verbose=1)
parameters = {
              'xgb__nthread':[64], #when use hyperthread, xgboost may become slower
              'xgb__objective':['binary:logistic'],
              'xgb__learning_rate': [0.01, 0.05], #so called `eta` value
              'xgb__max_depth': [10,15],
              'xgb__min_child_weight': [11],
              'xgb__silent': [1],
              'xgb__subsample': [0.8],
              'xgb__colsample_bytree': [0.7],
              'xgb__n_estimators': [1000, 3000], #number of trees, change it to 1000 for better results
              'xgb__missing':[-999],
              'xgb__seed': [seed]}


clf = GridSearchCV(model__, parameters, 
                   cv=10, 
                   scoring='mean_squared_error',
                   verbose=2)

In [0]:
#features = list(X.columns)
clf.fit(X, Y)

Fitting 10 folds for each of 8 candidates, totalling 80 fits
[CV] xgb__colsample_bytree=0.7, xgb__learning_rate=0.01, xgb__max_depth=10, xgb__min_child_weight=11, xgb__missing=-999, xgb__n_estimators=1000, xgb__nthread=64, xgb__objective=binary:logistic, xgb__seed=7, xgb__silent=1, xgb__subsample=0.8 


In [0]:
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print(':', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))


: -0.0006585220819181511
xgb__colsample_bytree: 0.7
xgb__learning_rate: 0.05
xgb__max_depth: 15
xgb__min_child_weight: 11
xgb__missing: -999
xgb__n_estimators: 500
xgb__nthread: 4
xgb__objective: 'binary:logistic'
xgb__seed: 7
xgb__silent: 1
xgb__subsample: 0.8


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [0]:
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print(':', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))


: -0.0006585220819181511
xgb__colsample_bytree: 0.7
xgb__learning_rate: 0.05
xgb__max_depth: 15
xgb__min_child_weight: 11
xgb__missing: -999
xgb__n_estimators: 500
xgb__nthread: 4
xgb__objective: 'binary:logistic'
xgb__seed: 7
xgb__silent: 1
xgb__subsample: 0.8


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [0]:
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print(':', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

: -0.0006583561225132363
xgb__colsample_bytree: 0.7
xgb__learning_rate: 0.05
xgb__max_depth: 15
xgb__min_child_weight: 11
xgb__missing: -999
xgb__n_estimators: 500
xgb__nthread: 4
xgb__objective: 'binary:logistic'
xgb__seed: 7
xgb__silent: 1
xgb__subsample: 0.8


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [0]:
for i in clf.grid_scores_: print(i)

mean: -0.03284, std: 0.00010, params: {'xgb__colsample_bytree': 0.7, 'xgb__learning_rate': 0.05, 'xgb__max_depth': 1, 'xgb__min_child_weight': 11, 'xgb__missing': -999, 'xgb__n_estimators': 20, 'xgb__nthread': 4, 'xgb__objective': 'binary:logistic', 'xgb__seed': 7, 'xgb__silent': 1, 'xgb__subsample': 0.8}

mean: -0.00228, std: 0.00008, params: {'xgb__colsample_bytree': 0.7, 'xgb__learning_rate': 0.05, 'xgb__max_depth': 1, 'xgb__min_child_weight': 11, 'xgb__missing': -999, 'xgb__n_estimators': 50, 'xgb__nthread': 4, 'xgb__objective': 'binary:logistic', 'xgb__seed': 7, 'xgb__silent': 1, 'xgb__subsample': 0.8}

mean: -0.00069, std: 0.00009, params: {'xgb__colsample_bytree': 0.7, 'xgb__learning_rate': 0.05, 'xgb__max_depth': 1, 'xgb__min_child_weight': 11, 'xgb__missing': -999, 'xgb__n_estimators': 500, 'xgb__nthread': 4, 'xgb__objective': 'binary:logistic', 'xgb__seed': 7, 'xgb__silent': 1, 'xgb__subsample': 0.8}

mean: -0.03284, std: 0.00010, params: {'xgb__colsample_bytree': 0.7, 'xgb__

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [0]:
# evaluate model with standardized dataset
prediction = clf.predict(X_test)

In [0]:
print(lbl.inverse_transform(numpy.array([[0.0006585220819181511]])))

[[0.04688677]]


In [0]:
# Run if 'continuous' error comes
prediction = lbl.inverse_transform([prediction])[0]

In [0]:
fh = open(base+'predictions_xgb.csv','w')
fh.write('ID,Prediction\n')
for i,yi in enumerate(prediction):
    fh.write('{},{}\n'.format(i,yi))
fh.close()

In [0]:
unique, counts = numpy.unique(prediction, return_counts=True)
print (numpy.asarray((unique, counts)).T)

[[ 0.03137269  1.        ]
 [ 0.03157933  1.        ]
 [ 0.03201855  1.        ]
 ...
 [21.20224419  2.        ]
 [21.87348981  1.        ]
 [24.19046679  1.        ]]
